In [1]:
# REMPLACER la cellule 1 par cette version améliorée
import os
import sys
import subprocess
import time
import traci
import numpy as np
import random
import sumolib
import warnings
warnings.filterwarnings('ignore')

# --- CONFIGURATION ROBUSTE ---
SUMO_FOLDER = "sumo_configs"
LOG_FOLDER = "simulation_logs"

# Création des dossiers
for folder in [SUMO_FOLDER, LOG_FOLDER]:
    if not os.path.exists(folder):
        os.makedirs(folder)

# Détection automatique de SUMO (Windows/Linux/Mac)
def find_sumo_path():
    possible_paths = [
        r"C:\\Program Files (x86)\\Eclipse\\Sumo",
        r"C:\\Sumo",
        r"/usr/local/share/sumo",
        r"/usr/share/sumo",
        os.path.expanduser("~/sumo")
    ]
    
    for path in possible_paths:
        if os.path.exists(path):
            return path
    raise FileNotFoundError("SUMO non trouvé. Installez-le ou spécifiez le chemin.")

# Configuration
if "SUMO_HOME" not in os.environ:
    try:
        os.environ["SUMO_HOME"] = find_sumo_path()
    except FileNotFoundError:
        os.environ["SUMO_HOME"] = input("Chemin SUMO_HOME non trouvé. Entrez le chemin: ")

# Ajout aux paths
sumo_tools = os.path.join(os.environ["SUMO_HOME"], "tools")
if sumo_tools not in sys.path:
    sys.path.append(sumo_tools)

sumo_bin = os.path.join(os.environ["SUMO_HOME"], "bin")
if sumo_bin not in os.environ["PATH"]:
    os.environ["PATH"] += os.pathsep + sumo_bin

print(f"✅ Environnement configuré. SUMO_HOME: {os.environ['SUMO_HOME']}")

✅ Environnement configuré. SUMO_HOME: C:\Program Files (x86)\Eclipse\Sumo\


In [3]:
# REMPLACER la cellule 2 par cette version corrigée
print("🔄 Génération du réseau avancée...")

# 1. Noeuds avec positions précises
nodes_xml = """<?xml version="1.0" encoding="UTF-8"?>
<nodes>
    <node id="C" x="0" y="0" type="traffic_light" tlType="static"/>
    <node id="N" x="0" y="150" type="priority"/>
    <node id="S" x="0" y="-150" type="priority"/>
    <node id="E" x="150" y="0" type="priority"/>
    <node id="W" x="-150" y="0" type="priority"/>
</nodes>"""

# 2. Routes avec voies dédiées (version simplifiée)
edges_xml = """<?xml version="1.0" encoding="UTF-8"?>
<edges>
    <!-- Nord vers Centre (2 voies + trottoir) -->
    <edge id="N2C" from="N" to="C" priority="1" numLanes="3" speed="13.89"/>
    
    <!-- Centre vers Sud -->
    <edge id="C2S" from="C" to="S" priority="1" numLanes="3" speed="13.89"/>
    
    <!-- Est vers Centre -->
    <edge id="E2C" from="E" to="C" priority="1" numLanes="3" speed="13.89"/>
    
    <!-- Centre vers Ouest -->
    <edge id="C2W" from="C" to="W" priority="1" numLanes="3" speed="13.89"/>
</edges>"""

# 3. Scénario enrichi (version améliorée)
routes_xml = """<?xml version="1.0" encoding="UTF-8"?>
<routes>
    <!-- Types de véhicules -->
    <vType id="car" accel="1.0" decel="4.5" length="4.3" minGap="2.5" 
           maxSpeed="15.0" sigma="0.5" guiShape="passenger"/>
    
    <vType id="bus" accel="0.8" decel="4.0" length="12.0" minGap="3.0"
           maxSpeed="10.0" guiShape="bus" color="0,128,0"/>
           
    <vType id="emergency" accel="3.0" decel="7.0" length="6.0" minGap="1.5"
           maxSpeed="25.0" guiShape="emergency" color="255,0,0"
           impatience="0.1"/>
           
    <vType id="pedestrian" vClass="pedestrian" width="0.8" length="0.5"
           minGap="0.5" maxSpeed="1.5" guiShape="person" color="255,255,0"/>
    
    <!-- Flux de véhicules -->
    <flow id="flow_N2S" type="car" begin="0" end="3600" period="5" 
          from="N2C" to="C2S" departLane="best"/>
    
    <flow id="flow_E2W" type="car" begin="0" end="3600" period="8"
          from="E2C" to="C2W" departLane="best"/>
    
    <!-- Bus régulier -->
    <flow id="bus_line" type="bus" begin="0" end="3600" period="120"
          from="N2C" to="C2S" departLane="1"/>
    
    <!-- Piétons -->
    <personFlow id="ped_flow_N" begin="0" end="3600" period="30">
        <walk from="N2C" to="C2S" departPos="0" arrivalPos="100"/>
    </personFlow>
    
    <personFlow id="ped_flow_E" begin="0" end="3600" period="40">
        <walk from="E2C" to="C2W" departPos="0" arrivalPos="100"/>
    </personFlow>
    
    <!-- Véhicules d'urgence programmés -->
    <vehicle id="ambulance_1" type="emergency" depart="150" departLane="1">
        <route edges="E2C C2W"/>
    </vehicle>
    
    <vehicle id="ambulance_2" type="emergency" depart="450" departLane="1">
        <route edges="N2C C2S"/>
    </vehicle>
</routes>"""

# 4. Configuration avec logging
config_xml = f"""<?xml version="1.0" encoding="UTF-8"?>
<configuration>
    <input>
        <net-file value="network.net.xml"/>
        <route-files value="routes.rou.xml"/>
    </input>
    
    <output>
        <fcd-output value="{os.path.join(LOG_FOLDER, 'trajectories.xml')}"/>
        <emission-output value="{os.path.join(LOG_FOLDER, 'emissions.xml')}"/>
    </output>
    
    <time>
        <begin value="0"/>
        <end value="2000"/>
        <step-length value="0.1"/>
    </time>
    
    <report>
        <verbose value="true"/>
        <no-step-log value="true"/>
    </report>
</configuration>"""

# Sauvegarde des fichiers
print("💾 Sauvegarde des fichiers de configuration...")
files_to_save = [
    ("nodes.nod.xml", nodes_xml),
    ("edges.edg.xml", edges_xml),
    ("routes.rou.xml", routes_xml),
    ("config.sumocfg", config_xml)
]

for filename, content in files_to_save:
    filepath = os.path.join(SUMO_FOLDER, filename)
    with open(filepath, "w", encoding="utf-8") as f:
        f.write(content)
    print(f"  ✅ {filename}")

# Compilation SIMPLIFIÉE et ROBUSTE
print("\n🔧 Compilation du réseau...")

# Première étape : Compilation de base
compile_cmd = [
    "netconvert",
    "--node-files", os.path.join(SUMO_FOLDER, "nodes.nod.xml"),
    "--edge-files", os.path.join(SUMO_FOLDER, "edges.edg.xml"),
    "--crossings.guess", "true",
    "--tls.guess", "true",
    "--no-turnarounds", "true",
    "-o", os.path.join(SUMO_FOLDER, "network.net.xml")
]

try:
    # Exécution avec capture d'erreurs
    result = subprocess.run(
        compile_cmd, 
        capture_output=True, 
        text=True, 
        shell=True,
        timeout=30  # Timeout de 30 secondes
    )
    
    if result.returncode == 0:
        print("✅ Réseau généré avec succès")
        if result.stderr:
            print(f"   ℹ️  Messages: {result.stderr[:100]}...")
    else:
        print(f"⚠️  Erreur lors de la compilation:")
        print(f"   Code erreur: {result.returncode}")
        print(f"   Sortie: {result.stdout[:200]}")
        print(f"   Erreurs: {result.stderr[:200]}")
        
        # Fallback : compilation minimale
        print("\n🔄 Tentative avec compilation minimale...")
        fallback_cmd = [
            "netconvert",
            "--node-files", os.path.join(SUMO_FOLDER, "nodes.nod.xml"),
            "--edge-files", os.path.join(SUMO_FOLDER, "edges.edg.xml"),
            "-o", os.path.join(SUMO_FOLDER, "network.net.xml")
        ]
        
        result_fallback = subprocess.run(
            fallback_cmd,
            capture_output=True,
            text=True,
            shell=True
        )
        
        if result_fallback.returncode == 0:
            print("✅ Réseau généré (version minimale)")
        else:
            print(f"❌ Échec de la compilation: {result_fallback.stderr[:200]}")
            print("\n💡 Solution possible:")
            print("1. Vérifiez que SUMO est correctement installé")
            print("2. Essayez: pip install sumolib")
            print("3. Vérifiez les permissions d'écriture")
            
except subprocess.TimeoutExpired:
    print("❌ Timeout: La compilation prend trop de temps")
except FileNotFoundError:
    print("❌ Erreur: netconvert non trouvé. Vérifiez l'installation de SUMO")
except Exception as e:
    print(f"❌ Erreur inattendue: {e}")

# Vérification finale
network_file = os.path.join(SUMO_FOLDER, "network.net.xml")
if os.path.exists(network_file) and os.path.getsize(network_file) > 1000:
    print(f"\n🎉 Réseau prêt! Taille: {os.path.getsize(network_file)} octets")
    
    # Aperçu du fichier généré
    try:
        with open(network_file, 'r', encoding='utf-8') as f:
            content = f.read(500)
            print(f"\n📄 Aperçu du réseau généré:")
            print("-" * 40)
            print(content)
            print("..." if len(content) == 500 else "")
            print("-" * 40)
    except:
        pass
else:
    print(f"\n⚠️  Fichier réseau non généré ou trop petit")
    
    # Vérification des fichiers d'entrée
    print("\n🔍 Vérification des fichiers d'entrée:")
    for filename in ["nodes.nod.xml", "edges.edg.xml"]:
        filepath = os.path.join(SUMO_FOLDER, filename)
        if os.path.exists(filepath):
            size = os.path.getsize(filepath)
            print(f"  ✅ {filename}: {size} octets")
        else:
            print(f"  ❌ {filename}: FICHIER MANQUANT")

# Instructions de débogage
print("\n🔧 Pour vérifier manuellement:")
print(f"cd {SUMO_FOLDER}")
print(f"netconvert --node-files nodes.nod.xml --edge-files edges.edg.xml --crossings.guess true -o network.net.xml")

🔄 Génération du réseau avancée...
💾 Sauvegarde des fichiers de configuration...
  ✅ nodes.nod.xml
  ✅ edges.edg.xml
  ✅ routes.rou.xml
  ✅ config.sumocfg

🔧 Compilation du réseau...
✅ Réseau généré avec succès

🎉 Réseau prêt! Taille: 9682 octets

📄 Aperçu du réseau généré:
----------------------------------------
<?xml version="1.0" encoding="UTF-8"?>

<!-- generated on 2025-12-18T13:28:52.160150+00:00 by Eclipse SUMO netconvert 1.25.0
<netconvertConfiguration xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://sumo.dlr.de/xsd/netconvertConfiguration.xsd">

    <input>
        <node-files value="sumo_configs\nodes.nod.xml"/>
        <edge-files value="sumo_configs\edges.edg.xml"/>
    </input>

    <output>
        <output-file value="sumo_configs\network.net.xml"/
...
----------------------------------------

🔧 Pour vérifier manuellement:
cd sumo_configs
netconvert --node-files nodes.nod.xml --edge-files edges.edg.xml --crossings.guess true -o n

In [4]:
# REMPLACER la cellule 3
class EnhancedTrafficAgent:
    def __init__(self, state_size=16, action_size=2):
        self.state_size = state_size
        self.action_size = action_size
        
        # Paramètres d'apprentissage adaptatifs
        self.alpha_start = 0.2  # Learning rate initial
        self.alpha = self.alpha_start
        self.alpha_min = 0.01
        self.alpha_decay = 0.995
        
        self.gamma = 0.95  # Discount factor
        
        self.epsilon_start = 0.3  # Exploration initiale
        self.epsilon = self.epsilon_start
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.998
        
        # Q-Table avec initialisation optimiste
        self.q_table = {}
        self.initial_q_value = 1.0  # Optimistic initialization
        
        # Statistiques
        self.stats = {
            'episodes': 0,
            'total_reward': 0,
            'actions_taken': {0: 0, 1: 0},
            'exploration_rate': []
        }
        
        # Mémoire pour l'apprentissage par lots
        self.memory = []
        self.batch_size = 32
    
    def encode_state(self, density_N, density_E, waiting_time_N, waiting_time_E, has_pedestrians):
        """
        État enrichi avec:
        - Densité des véhicules (0-3)
        - Temps d'attente moyen (0-2)
        - Présence de piétons (0-1)
        """
        def classify_density(n):
            if n == 0: return 0
            if n <= 2: return 1
            if n <= 5: return 2
            return 3
            
        def classify_waiting_time(t):
            if t < 10: return 0
            if t < 30: return 1
            return 2
        
        density_state = (classify_density(density_N), classify_density(density_E))
        waiting_state = (classify_waiting_time(waiting_time_N), classify_waiting_time(waiting_time_E))
        pedestrian_state = 1 if has_pedestrians else 0
        
        # Combinaison en un seul état (peut être hashé)
        state_key = f"{density_state[0]}{density_state[1]}{waiting_state[0]}{waiting_state[1]}{pedestrian_state}"
        return state_key
    
    def initialize_state(self, state):
        """Initialisation optimiste de l'état"""
        if state not in self.q_table:
            self.q_table[state] = [self.initial_q_value] * self.action_size
    
    def choose_action(self, state):
        """Epsilon-greedy avec décroissance"""
        self.initialize_state(state)
        
        # Décroissance de l'exploration
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)
        self.alpha = max(self.alpha_min, self.alpha * self.alpha_decay)
        
        if random.random() < self.epsilon:
            # Exploration: parfois favoriser la voie la plus chargée
            if random.random() < 0.7:
                # 70% du temps: choisir intelligemment
                if state[0] > state[2]:  # Plus de voitures au Nord
                    action = 0
                else:
                    action = 1
            else:
                # 30%: exploration pure
                action = random.choice([0, 1])
            exp_type = "exploration"
        else:
            # Exploitation: meilleure action connue
            q_values = self.q_table[state]
            max_q = max(q_values)
            # En cas d'égalité, choisir aléatoirement parmi les meilleures
            best_actions = [i for i, q in enumerate(q_values) if q == max_q]
            action = random.choice(best_actions)
            exp_type = "exploitation"
        
        self.stats['actions_taken'][action] += 1
        self.stats['exploration_rate'].append(self.epsilon)
        
        return action, exp_type
    
    def compute_reward(self, densities, waiting_times, action_taken, has_emergency, pedestrians_waiting):
        """
        Récompense multi-objectifs:
        1. Réduire le nombre total de véhicules en attente
        2. Réduire le temps d'attente
        3. Prioriser les urgences
        4. Considérer les piétons
        """
        base_reward = -0.1  # Petite pénalité constante pour encourager l'action
        
        # 1. Densité des véhicules (pénalité quadratique)
        density_penalty = -0.05 * (densities[0]**2 + densities[1]**2)
        
        # 2. Temps d'attente (pénalité linéaire)
        waiting_penalty = -0.01 * (waiting_times[0] + waiting_times[1])
        
        # 3. Bonus si action correspond à la voie la plus chargée
        if action_taken == 0 and densities[0] > densities[1]:
            congestion_bonus = 0.5
        elif action_taken == 1 and densities[1] > densities[0]:
            congestion_bonus = 0.5
        else:
            congestion_bonus = 0.0
        
        # 4. Pénalité si piétons attendent trop longtemps
        pedestrian_penalty = -0.2 * pedestrians_waiting if pedestrians_waiting > 3 else 0
        
        # 5. Bonus si urgence traitée
        emergency_bonus = 2.0 if has_emergency else 0
        
        total_reward = (base_reward + density_penalty + waiting_penalty + 
                       congestion_bonus + pedestrian_penalty + emergency_bonus)
        
        return total_reward
    
    def learn(self, state, action, reward, next_state):
        """Q-Learning avec mémoïsation"""
        self.initialize_state(next_state)
        
        # Formule Q-Learning standard
        old_value = self.q_table[state][action]
        next_max = max(self.q_table[next_state])
        
        new_value = old_value + self.alpha * (reward + self.gamma * next_max - old_value)
        self.q_table[state][action] = new_value
        
        # Mettre à jour les statistiques
        self.stats['total_reward'] += reward
        
        # Stocker dans la mémoire pour apprentissage par lots
        self.memory.append((state, action, reward, next_state))
        if len(self.memory) > 1000:  # Limite de mémoire
            self.memory.pop(0)
    
    def batch_learn(self):
        """Apprentissage sur un batch d'expériences passées"""
        if len(self.memory) < self.batch_size:
            return
            
        batch = random.sample(self.memory, self.batch_size)
        for state, action, reward, next_state in batch:
            self.learn(state, action, reward, next_state)
    
    def save_model(self, filename="q_learning_model.pkl"):
        """Sauvegarde du modèle"""
        import pickle
        with open(filename, 'wb') as f:
            pickle.dump({
                'q_table': self.q_table,
                'stats': self.stats,
                'epsilon': self.epsilon,
                'alpha': self.alpha
            }, f)
    
    def load_model(self, filename="q_learning_model.pkl"):
        """Chargement du modèle"""
        import pickle
        try:
            with open(filename, 'rb') as f:
                data = pickle.load(f)
                self.q_table = data['q_table']
                self.stats = data['stats']
                self.epsilon = data['epsilon']
                self.alpha = data['alpha']
        except FileNotFoundError:
            print("Aucun modèle précédent trouvé. Démarrage frais.")

In [5]:
# REMPLACER la cellule 4
class TrafficDetector:
    """Système complet de détection multi-modale"""
    
    @staticmethod
    def detect_vehicles():
        """Détection des véhicules avec types"""
        edges = ["N2C", "E2C"]
        results = {}
        
        for edge in edges:
            vehicle_ids = traci.edge.getLastStepVehicleIDs(edge)
            vehicles = []
            
            for vid in vehicle_ids:
                vtype = traci.vehicle.getTypeID(vid)
                speed = traci.vehicle.getSpeed(vid)
                waiting_time = traci.vehicle.getWaitingTime(vid)
                
                vehicles.append({
                    'id': vid,
                    'type': vtype,
                    'speed': speed,
                    'waiting_time': waiting_time,
                    'is_stopped': speed < 0.1
                })
            
            results[edge] = {
                'count': len(vehicle_ids),
                'halted': traci.edge.getLastStepHaltingNumber(edge),
                'vehicles': vehicles,
                'mean_speed': traci.edge.getLastStepMeanSpeed(edge),
                'occupancy': traci.edge.getLastStepOccupancy(edge)
            }
        
        return results
    
    @staticmethod
    def detect_pedestrians():
        """Détection des piétons"""
        pedestrian_ids = traci.person.getIDList()
        waiting_pedestrians = 0
        
        for pid in pedestrian_ids:
            # Vérifier si le piéton attend à un passage
            if traci.person.getWaitingTime(pid) > 5:  # Attend depuis plus de 5 secondes
                waiting_pedestrians += 1
        
        return {
            'total': len(pedestrian_ids),
            'waiting': waiting_pedestrians,
            'waiting_time': max([traci.person.getWaitingTime(pid) for pid in pedestrian_ids] + [0])
        }
    
    @staticmethod
    def detect_emergencies():
        """Détection spécifique des véhicules d'urgence"""
        all_vehicles = traci.vehicle.getIDList()
        emergencies = []
        
        for vid in all_vehicles:
            if traci.vehicle.getTypeID(vid) == "emergency":
                # Vérifier la proximité de l'intersection
                pos = traci.vehicle.getLanePosition(vid)
                lane = traci.vehicle.getLaneID(vid)
                
                if any(edge in lane for edge in ["N2C", "E2C"]):
                    emergencies.append({
                        'id': vid,
                        'lane': lane,
                        'distance_to_intersection': 100 - pos if pos < 100 else 0,
                        'speed': traci.vehicle.getSpeed(vid)
                    })
        
        return emergencies
    
    @staticmethod
    def calculate_waiting_times():
        """Calcule les temps d'attente par direction"""
        edges = ["N2C", "E2C"]
        waiting_times = {}
        
        for edge in edges:
            vehicles = traci.edge.getLastStepVehicleIDs(edge)
            if vehicles:
                total_wait = sum(traci.vehicle.getWaitingTime(vid) for vid in vehicles)
                avg_wait = total_wait / len(vehicles)
            else:
                avg_wait = 0
            
            waiting_times[edge] = avg_wait
        
        return waiting_times

In [ ]:
# --- CELLULE 5 : RÉPARATION TOTALE ET SIMULATION ---
# Cette cellule régénère tout de A à Z pour éviter les fichiers corrompus.

import os
import sys
import time
import traci
import subprocess
import random
import numpy as np
import matplotlib.pyplot as plt

# CONFIGURATION
SUMO_FOLDER = "sumo_configs"
if not os.path.exists(SUMO_FOLDER):
    os.makedirs(SUMO_FOLDER)

# ==========================================
# ÉTAPE 1 : GÉNÉRATION DES FICHIERS (CLEAN INSTALL)
# ==========================================
def regenerate_files():
    print("♻️  Régénération des fichiers de configuration...")
    
    # 1. Définition des Noeuds
    nodes_xml = """<nodes>
    <node id="C" x="0" y="0" type="traffic_light"/>
    <node id="N" x="0" y="200" type="priority"/>
    <node id="S" x="0" y="-200" type="priority"/>
    <node id="E" x="200" y="0" type="priority"/>
    <node id="W" x="-200" y="0" type="priority"/>
</nodes>"""

    # 2. Définition des Routes
    edges_xml = """<edges>
    <edge id="N2C" from="N" to="C" priority="1" numLanes="2" speed="13.89"/>
    <edge id="C2S" from="C" to="S" priority="1" numLanes="2" speed="13.89"/>
    <edge id="E2C" from="E" to="C" priority="1" numLanes="2" speed="13.89"/>
    <edge id="C2W" from="C" to="W" priority="1" numLanes="2" speed="13.89"/>
</edges>"""

    # 3. Flux de trafic
    routes_xml = """<routes>
    <vType id="car" accel="1.0" decel="4.5" length="5.0" minGap="2.5" maxSpeed="15.0" guiShape="passenger"/>
    <flow id="flow_N" type="car" begin="0" end="3600" period="3" from="N2C" to="C2S"/>
    <flow id="flow_E" type="car" begin="0" end="3600" period="4" from="E2C" to="C2W"/>
</routes>"""

    # 4. Configuration SUMO
    config_xml = f"""<configuration>
    <input>
        <net-file value="network.net.xml"/>
        <route-files value="routes.rou.xml"/>
    </input>
    <time>
        <begin value="0"/>
        <end value="2000"/>
    </time>
    <report>
        <verbose value="true"/>
        <no-step-log value="true"/>
    </report>
</configuration>"""

    # Écriture des fichiers
    with open(os.path.join(SUMO_FOLDER, "nodes.nod.xml"), "w") as f: f.write(nodes_xml)
    with open(os.path.join(SUMO_FOLDER, "edges.edg.xml"), "w") as f: f.write(edges_xml)
    with open(os.path.join(SUMO_FOLDER, "routes.rou.xml"), "w") as f: f.write(routes_xml)
    with open(os.path.join(SUMO_FOLDER, "config.sumocfg"), "w") as f: f.write(config_xml)

    # Compilation du réseau (netconvert)
    print("🔧 Compilation du réseau (netconvert)...")
    cmd = [
        "netconvert",
        "--node-files", os.path.join(SUMO_FOLDER, "nodes.nod.xml"),
        "--edge-files", os.path.join(SUMO_FOLDER, "edges.edg.xml"),
        "-o", os.path.join(SUMO_FOLDER, "network.net.xml"),
        "--no-turnarounds", "true"
    ]
    
    try:
        result = subprocess.run(cmd, capture_output=True, text=True, check=True)
        print("✅ Réseau compilé avec succès.")
    except subprocess.CalledProcessError as e:
        print(f"❌ ERREUR NETCONVERT: {e.stderr}")
        return False
    except FileNotFoundError:
        print("❌ ERREUR: Commande 'netconvert' introuvable. Vérifiez l'installation de SUMO.")
        return False
        
    return True

# ==========================================
# ÉTAPE 2 : AGENT INTELLIGENT
# ==========================================
class TrafficAgent:
    def __init__(self):
        self.q_table = {}
        self.epsilon = 0.1
        self.alpha = 0.1
        self.gamma = 0.9
        self.last_state = None
        self.last_action = None

    def get_state(self, n, e):
        # 0=Vide, 1=Peu, 2=Moyen, 3=Plein
        def d(x): return 0 if x==0 else 1 if x<4 else 2 if x<7 else 3
        return (d(n), d(e))

    def choose_action(self, state):
        if state not in self.q_table: self.q_table[state] = [0.0, 0.0]
        if random.random() < self.epsilon: return random.choice([0, 1])
        return np.argmax(self.q_table[state])

    def learn(self, state, action, reward, next_state):
        if state not in self.q_table: self.q_table[state] = [0.0, 0.0]
        if next_state not in self.q_table: self.q_table[next_state] = [0.0, 0.0]
        old = self.q_table[state][action]
        mx = np.max(self.q_table[next_state])
        self.q_table[state][action] = (1-self.alpha)*old + self.alpha*(reward + self.gamma*mx)

# ==========================================
# ÉTAPE 3 : DIAGNOSTIC & LANCEMENT
# ==========================================
def run_simulation_robust():
    # 1. Régénérer les fichiers pour être sûr
    if not regenerate_files():
        return

    # 2. Nettoyage
    try: traci.close()
    except: pass
    
    if os.name == 'nt':
        os.system("taskkill /f /im sumo.exe >nul 2>&1")
    else:
        os.system("pkill -f sumo")
        
    time.sleep(1)

    # 3. Test de pré-lancement (Diagnostic)
    config_path = os.path.join(SUMO_FOLDER, "config.sumocfg")
    print("\n🩺 Diagnostic de pré-lancement...")
    test_cmd = ["sumo", "-c", config_path, "--start", "--quit-on-end"]
    test_proc = subprocess.run(test_cmd, capture_output=True, text=True)
    
    if test_proc.returncode != 0:
        print("❌ SUMO CRASH AU DÉMARRAGE ! Voici l'erreur exacte :")
        print("-" * 40)
        print(test_proc.stderr)
        print("-" * 40)
        return

    # 4. Lancement réel
    print("🚀 Démarrage de la simulation...")
    use_gui = True # Mettre à True si ça marche enfin
    binary = "sumo-gui" if use_gui else "sumo"
    port = 8813
    
    cmd = [binary, "-c", config_path, "--start", "--quit-on-end"]
    if use_gui: cmd += ["--delay", "50"]
    
    try:
        traci.start(cmd, port=port)
        print("✅ Connexion Traci établie !")
        
        # --- BOUCLE PRINCIPALE ---
        agent = TrafficAgent()
        stats_n, stats_e = [], []
        
        for step in range(500):
            traci.simulationStep()
            
            n = traci.edge.getLastStepHaltingNumber("N2C")
            e = traci.edge.getLastStepHaltingNumber("E2C")
            
            # IA
            state = agent.get_state(n, e)
            reward = -(n + e)
            if agent.last_state:
                agent.learn(agent.last_state, agent.last_action, reward, state)
            
            action = agent.choose_action(state)
            
            # Feux (0=Nord Vert, 2=Est Vert)
            phase = 0 if action == 0 else 2
            if traci.trafficlight.getPhase("C") != phase:
                traci.trafficlight.setPhase("C", phase)
                
            agent.last_state = state
            agent.last_action = action
            
            stats_n.append(n)
            stats_e.append(e)
            
            if step % 50 == 0:
                print(f"   Step {step}: Nord={int(n)} Est={int(e)}")
                
        print("🏁 Simulation terminée.")
        traci.close()
        
        # Plot
        plt.plot(stats_n, label="Nord"); plt.plot(stats_e, label="Est")
        plt.legend(); plt.show()
        
    except Exception as e:
        print(f"❌ Erreur pendant l'exécution: {e}")
        try: traci.close()
        except: pass

if __name__ == "__main__":
    run_simulation_robust()

🚦 SYSTÈME DE CONTRÔLE DE TRAFIC INTELLIGENT
🤖 Mode: Q-Learning + Urgences + Anti-blocage


NameError: name 'TrafficDetector' is not defined

: 